# 1. Import Required Libraries and Set Parameters
This cell imports the necessary libraries, sets up warning filters, and defines the year and system to be used in the analysis.

In [3]:
import os
import mario
import warnings
filters = warnings.filterwarnings("ignore")

year = 2020
system = "ixi"

# 2. Download EXIOBASE Data
This cell downloads the EXIOBASE data for the specified year and system, saving it to the appropriate directory.

In [ ]:
import requests
import os

url = f"https://zenodo.org/records/14869924/files/IOT_{year}_{system}.zip?download=1"

# Create the directory if it doesn't exist
output_dir = "Data/Database"
os.makedirs(output_dir, exist_ok=True)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    file_path = os.path.join(output_dir, f"IOT_{year}_{system}.zip")
    with open(file_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved to {file_path}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


# 3. Parse EXIOBASE Data
This cell parses the downloaded EXIOBASE data into a database object for further processing.

In [ ]:
db = mario.parse_exiobase(
    path=f"Data/Database/IOT_{year}_{system}.zip", 
    table='IOT',
    unit='Monetary')

# 4. Prepare Aggregation Files
This cell creates the aggregation directory, generates a blank aggregation Excel file, and performs aggregation if the file is available.

In [ ]:
aggr_dir = "Data/Aggregations"
os.makedirs(aggr_dir, exist_ok=True)

db.get_aggregation_excel(f"{aggr_dir}/Aggregation_blank.xlsx")
db.aggregate(f"{aggr_dir}/Aggregation.xlsx", ignore_nan = True)

# 5. Export Baseline Data
This cell creates the baseline directory and exports the parsed and aggregated data as text files for future use.

In [ ]:
baseline_dir = "Data/Database/Baseline"

os.makedirs(f"Data/Database/Baseline/{year}_{system}", exist_ok=True)
db.to_txt(f"{baseline_dir}/{year}_{system}")

# 6. Load Baseline Data (Optional)
If the baseline data has already been generated, this cell shows how to reload it from the text files.

In [4]:
# If you already generated the baseline, start from here
baseline_dir = "Data/Database/Baseline"

db = mario.parse_from_txt(
    path = f"{baseline_dir}/{year}_{system}/flows",
    table = "IOT",
    mode = "flows",
)

# 7. Prepare Shock Files
This cell creates the shocks directory and generates a blank shock Excel file for scenario analysis.

In [5]:
shock_dir = "Data/Shocks"
os.makedirs(shock_dir, exist_ok=True)

db.get_shock_excel(f"{shock_dir}/Shock_blank.xlsx")

# 8. Run Scenario Analysis
This cell defines scenarios and runs the shock calculations for each scenario using the prepared shock files.

In [6]:
scenarios = {
    'scenario 1_t': {"Y":True, "z":True},
}

for scenario,info in scenarios.items():
   db.shock_calc(io=f'{shock_dir}/{scenario}.xlsx',Y=info["Y"],z=info["z"],scenario=scenario, force_rewrite=True)

# 9. Export Results
This cell creates the results directory and exports the results of the scenario analysis to SQL and Excel files.

In [7]:
results_dir = f"Results/{year}_{system}"
os.makedirs(results_dir, exist_ok=True)

db.to_sql(
    path = results_dir,
    matrices = ["E","Y","V"],
    sets_to_excel=True,
    mapping_cols = 2,
)


Matrix: E | Sub-matrix: E | Scenario: baseline... Scenario: scenario 1_t... 
Matrix: Y | Sub-matrix: Y | Scenario: baseline... Scenario: scenario 1_t... 
Matrix: V | Sub-matrix: V | Scenario: baseline... Scenario: scenario 1_t... 

# End of Workflow
The workflow is now complete. You can proceed to analyze the results or visualize them as needed.